# The Wealth of Nations
# Links Between Economic Health and Environmental Welfare

## 1. Data analysis and collection
Exploration of available databases from World Bank API and selection of indicators

In [ ]:
import wbgapi as wb
import pandas as pd
print(wb.source.info())

economies = list(wb.economy.list())
print(">>>>>>> NUMBER OF ECONOMIES IN THE DATABASE >>>>>>>")
print(len(economies))
print(economies[:5])


Finding the indicators related to the economy such as GDP or GDP growth.

In [ ]:
WDI = 2     # second database = WDI (World Development Indicators)

for ind in wb.series.list(db = WDI, q = ('gdp')):
    print(f"{ind['id']}: {ind['value']}")


Looking over health indicators such as life expectancy, mortality and health expenditure which links economy to health of the population.

In [ ]:
for query in ['life expectancy', 'mortality', 'health expenditure']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")


Examining possible indicators for environmental welfare for a country focusing on pollution, use of renewable energy or access to clean water.

In [ ]:
for query in ['CO2', 'pollution', 'greenhouse', 'renewable energy', 'forest', 'water']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")


Exploring population its growth and urbanization to see how it affects the economy, health, and environment.

In [ ]:
for query in ['population', 'urban']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")

## Choosing main indicators for each group
### Economy 
 - GDP per capita, PPP (constant 2021 international $) — measures average economic output per person.
 - GDP growth (annual %) — tracks how fast the economy is expanding/contracting over time.
 - Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP) — shows how much CO2 is emitted per unit of economic output, linking economic activity to environmental cost.

### Health
 - Life expectancy at birth, total (years) — average expected lifespan at birth, measure of overall health.
 - Mortality rate, under-5 (per 1,000 live births) — deaths of children under five, demonstrates healthcare access and quality.
 - Current health expenditure (% of GDP) — investment in health sector.

-- Indicators which I found under environment indicators explorations but are related to health sector and will provide meaningful links between these sectors, connects environmental conditions to human health.
 - Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)connects environmental conditions to human health.
 - Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)

### Environment
 - Carbon dioxide (CO2) emissions (total) excluding LULUCF (Mt CO2e)
 - Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)CO2 per capita 
 - PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) — indicator of air quality.
 - Total greenhouse gas emissions excluding LULUCF (Mt CO2e)
 - Total greenhouse gas emissions excluding LULUCF per capita (t CO2e/capita)
 - Renewable energy consumption (% of total) — sustainability metric.
 - Forest area (% of land area) —  air quality, biodiversity.
 - People using safely managed drinking water services (% of population) — access to clean drinking water.

### Population and Urbanization Indicators
 - Population growth (annual %)
 - Population, total
 - Urban population (% of total population) — share of people living in cities, reflecting urbanization level.
 - Population density (people per sq. km of land area) — influencing environmental stress.

Next steps: selecting the countries, pulling the data, checking the missing values, possibly adjusting number of indicators based on coverage.

## Country selection
1. Selecting the real countries - filtering out economies in the list named such as Africa Eastern and Southern, World...
2. Keeping the sample of countries limited to larger economies - filtering out small countries with population under 5 millions

In [ ]:
for e in wb.economy.list(): 
    if e['aggregate'] == False:
        print(f"{e['id']}: {e['value']}")

In [ ]:
non_aggregates = [e for e in wb.economy.list() if e['aggregate'] == False]
print(len(non_aggregates))
economy_ids = [e['id'] for e in non_aggregates]

# Filtering economies with population > 5 million
min_pop = 5000000  # minimum population
pop_data = wb.data.DataFrame('SP.POP.TOTL', economy_ids)
print(pop_data[:3])

# Selecting countries satisfying the population criterion for the last available year
filtered_countries = pop_data.index[pop_data.iloc[:, -1] >= min_pop].tolist()
print(filtered_countries[:10])
print(len(filtered_countries))


After filtering, I am left with 126 countries. This set is sufficient to capture global trends while reducing the number of very small economies, which helps save computing time. In the data_collection.py I collect all the data for chosen indicators and countries.

In [ ]:
df = pd.read_csv("/Users/jarka/Desktop/Python-project/data/raw/all_indicators.csv")
print(df.head())
print(df.shape)


In [96]:
year_cols = df.columns[2:]

# Sort indicators by missing fraction in descending order
missing_per_indicator = df.groupby('series')[year_cols].apply(lambda x: x.isna().mean().mean())
print("Missing values per indicator:")
print(missing_per_indicator.sort_values(ascending=False))

Missing values per indicator:
series
SH.STA.WASH.P5             0.920635
SH.STA.AIRP.P5             0.920635
SH.H2O.SMDW.ZS             0.307937
EN.ATM.PM25.MC.M3          0.126667
EG.FEC.RNEW.ZS             0.078095
SH.XPD.CHEX.PC.CD          0.070794
SH.XPD.CHEX.GD.ZS          0.070159
EN.GHG.CO2.RT.GDP.PP.KD    0.053333
NY.GDP.PCAP.PP.KD          0.038095
EN.GHG.ALL.MT.CE.AR5       0.022857
EN.GHG.ALL.PC.CE.AR5       0.022857
EN.GHG.CO2.MT.CE.AR5       0.022857
EN.GHG.CO2.PC.CE.AR5       0.022857
NY.GDP.MKTP.KD.ZG          0.017778
AG.LND.FRST.ZS             0.017143
EN.POP.DNST                0.009524
SH.DYN.MORT                0.007619
SP.DYN.LE00.IN             0.000000
SP.POP.GROW                0.000000
SP.POP.TOTL                0.000000
SP.URB.TOTL.IN.ZS          0.000000
dtype: float64
